In [1]:
import boto3
from pyspark.sql.functions import col, datediff, current_timestamp, expr, substring, lpad, row_number, desc, to_date, lead, lag
from pyspark.sql import Window

Calculation started (calculation_id=6cca5e00-6845-9f32-ac40-7c9bbf59c7e7) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [2]:
def list_parquet_files(bucket_name, prefix):
    # List objects in the specified S3 bucket
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    parquet_files = [] 
    # Check if the bucket contains any objects
    if 'Contents' in response:
        # Iterate through the list of objects and filter for parquet files
        for obj in response['Contents']:
            if obj['Key'].endswith('.parquet'):
                parquet_files.append("s3://" + bucket_name + "/"+ obj['Key'])
        return parquet_files           
    
    else:
        print("No files found in the bucket.")

Calculation started (calculation_id=c0ca5e00-6e03-b398-a4a0-cb5035be7262) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
def read_parquet(files):
    if isinstance(files, list):
        df = spark.read.parquet(*files)
    else:
        df = spark.read.parquet(files)
    return df

Calculation started (calculation_id=2aca5e00-7301-8f0c-cb08-d9e134678eff) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [4]:
def read_csv(file_path):
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    return df

Calculation started (calculation_id=0aca5e00-783d-891b-6e3f-d7455756cb47) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [5]:
def write_to_delta(source, target, partition_cols):
    source.write.format("delta", mode="overwrite").partitionBy(*partition_cols).save(target)
    return

Calculation started (calculation_id=16ca5e00-795d-da70-1376-497a39bd68e9) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [6]:
from delta.tables import DeltaTable

def write_to_target(spark, updates, target, partition_cols, merge_condition):
    if DeltaTable.isDeltaTable(spark, target):
        deltaTable = DeltaTable.forPath(spark, target)
        deltaTable.alias("target") \
                .merge(updates.alias("updates"), merge_condition) \
                .whenMatchedUpdateAll() \
                .whenNotMatchedInsertAll() \
                .execute()
    else:
        write_to_delta(updates, target, partition_cols)

Calculation started (calculation_id=f8ca5e00-7e5f-dd1c-69fa-06eecb128049) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation f8ca5e00-7e5f-dd1c-69fa-06eecb128049 failed


  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'delta'



In [7]:
bucket = "datacraft-retail-dataset"
stage_prefix = "stage/data/"

Calculation started (calculation_id=04ca5e00-836a-b36e-b19a-0a93bb225074) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [8]:
# Initialize an S3 client
s3_client = boto3.client('s3')

# Call the function to list parquet files
parquet_files = list_parquet_files(bucket, stage_prefix)

if parquet_files:
    print("Parquet files in the bucket:")
    for file in parquet_files:
        print(file)
else:
    print("No Parquet files found.")

Calculation started (calculation_id=6cca5e00-887c-52f3-0bd6-5a408ad94cb8) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Parquet files in the bucket:
s3://datacraft-retail-dataset/stage/data/2025/01/28/12/retail-delivery-stream-1-2025-01-28-12-41-24-6fb1bafc-974d-4f91-a63c-c5596847437f.parquet
s3://datacraft-retail-dataset/stage/data/2025/01/28/12/retail-delivery-stream-1-2025-01-28-12-43-25-dd25f467-dd69-4d11-8747-441fb8959734.parquet
s3://datacraft-retail-dataset/stage/data/2025/01/28/12/retail-delivery-stream-1-2025-01-28-12-44-25-18136aa3-1d66-4738-b689-0dd936b351ff.parquet
s3://datacraft-retail-dataset/stage/data/2025/01/28/12/retail-delivery-stream-1-2025-01-28-12-45-26-84346549-5017-4d33-a143-db31c5ef5a03.parquet
s3://datacraft-retail-dataset/stage/data/2025/01/28/12/retail-delivery-stream-1-2025-01-28-12-46-29-58257fcb-bf85-451e-bbc7-52e519aba4e4.parquet
s3://datacraft-retail-dataset/stage/data/2025/01/28/12/retail-delivery-stream-1-2025-01-28-12-48-25-c9fbb564-6e90-409a-9adf-5ee8918a22b5.parquet
s3://datacraft-retail-dataset/stage/data/2025/01/28/12/retail-delivery-stream-

In [9]:
df_stage = read_parquet(parquet_files)

Calculation started (calculation_id=46ca5e00-8dad-a487-de16-538ccbe46210) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [10]:
df_stage.show(10)
df_stage.printSchema()

Calculation started (calculation_id=e2ca5e00-9ec8-c1d3-760a-4dcaf4c519f2) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------------------+-----------------------------+---------------------+----------------------+--------------------------+--------------------------------+----------------------------------+-------------------------------+--------------------------------+--------------------------------+---------------------------------+------------------------------+--------------------------------+--------------------------------+---------------------------------+-------------------------------+------------------------------+----------------------------+------------------------------+------------------------------+-----------------------------------+--------------------+-----------------------------+
|retailtxndata_invoiceno|retailtxndata_invoicedatetime|retailtxndata_storeno|retailtxndata_storepos|retailtxndata_storecountry|retailtxndata_lineitem_stockcode|retailtxndata_lineitem_description|retailtxndata_lineitem_quantity|retailtxndata_lineitem_unitprice|retailtxndata_lin

In [11]:
df_stage.createOrReplaceTempView("df_stage")

Calculation started (calculation_id=9eca5e00-afcc-d085-df3a-2abefa319329) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [12]:
df_clean =  spark.sql(""" select cast(trim(RetailTxnData_InvoiceNo) as long) as invoiceno,
                        cast(trim(RetailTxnData_InvoiceDateTime) as timestamp) as invoicets,
                        date(cast(trim(RetailTxnData_InvoiceDateTime) as timestamp)) as invoicedate,
                        year(cast(trim(RetailTxnData_InvoiceDateTime) as timestamp)) as invoice_year,
                        month(cast(trim(RetailTxnData_InvoiceDateTime) as timestamp)) as invoice_month,
                        cast(trim(RetailTxnData_StoreNo) as string) as storeno,
                        cast(trim(RetailTxnData_StorePos) as string) as storepos,
                        cast(trim(RetailTxnData_StoreCountry) as string) as storecountry,
                        cast(trim(RetailTxnData_LineItem_StockCode) as string) as lineitem_stockcode,
                        cast(trim(RetailTxnData_LineItem_Description) as string) as lineitem_description,
                        cast(trim(RetailTxnData_LineItem_Quantity) as double) as lineitem_quantity,
                        cast(trim(REPLACE(RetailTxnData_LineItem_UnitPrice, '$', '')) as float) as lineitem_unitprice,
                        cast(trim(REPLACE(RetailTxnData_LineItem_SalePrice, '$', '')) as float) as lineitem_saleprice,
                        cast(trim(RetailTxnData_Customer_CustomerID) as long) as customer_customerid,
                        cast(trim(RetailTxnData_Customer_EmailId) as string) as customer_emailid,
                        cast(trim(RetailTxnData_Customer_ContactNo) as string) as customer_contactno,
                        cast(trim(RetailTxnData_Customer_FirstName) as string) as customer_firstname,
                        cast(trim(RetailTxnData_Customer_MiddleName) as string) as customer_middlename,
                        cast(trim(RetailTxnData_Customer_LastName) as string) as customer_lastname,
                        cast(trim(RetailTxnData_Customer_Address) as string) as customer_address,
                        cast(trim(RetailTxnData_Customer_State) as string) as customer_state,
                        cast(trim(RetailTxnData_Customer_Country) as string) as customer_country,
                        cast(trim(RetailTxnData_Customer_ZipCode) as string) as customer_zipcode,
                        cast(trim(RetailTxnData_Discount_DiscountCode) as string) as discountcode,
                        cast(trim(RetailTxnData_Region) as string) as region,
                        cast(trim(RetailTxnData_SourceTimestamp) as timestamp) as sourcets
                        from df_stage""")

df_clean.printSchema()

Calculation started (calculation_id=04ca5e00-b4cb-32af-cbe2-5c530de68611) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
root
 |-- invoiceno: long (nullable = true)
 |-- invoicets: timestamp (nullable = true)
 |-- invoicedate: date (nullable = true)
 |-- invoice_year: integer (nullable = true)
 |-- invoice_month: integer (nullable = true)
 |-- storeno: string (nullable = true)
 |-- storepos: string (nullable = true)
 |-- storecountry: string (nullable = true)
 |-- lineitem_stockcode: string (nullable = true)
 |-- lineitem_description: string (nullable = true)
 |-- lineitem_quantity: double (nullable = true)
 |-- lineitem_unitprice: float (nullable = true)
 |-- lineitem_saleprice: float (nullable = true)
 |-- customer_customerid: long (nullable = true)
 |-- customer_emailid: string (nullable = true)
 |-- customer_contactno: string (nullable = true)
 |-- customer_firstname: string (nullable = true)
 |-- customer_middlename: string (nullable = true)
 |-- customer_lastname: string (nullable = true)
 |-- customer_address: string (nullable = true)
 |-- customer_state: string (nullable = 

In [13]:
# Handling nulls
fill_numeric_cols = {
    "invoiceno":-1,
    "customer_customerid":-1
    }
fill_floating_cols = {
    "lineitem_quantity":"NaN",
    "lineitem_unitprice":"NaN",
    "lineitem_saleprice":"NaN"
    }
fill_timestamp_cols = {
    "invoicets":"1900-01-01 00:00:00",
    "sourcets":"1900-01-01 00:00:00"
    }

df_clean = df_clean.na.fill("NaN") \
    .na.fill(fill_numeric_cols) \
    .na.fill(fill_floating_cols) \
    .na.fill(fill_timestamp_cols)

Calculation started (calculation_id=1aca5e00-b9f3-ee93-afd5-83f1794f1ef4) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [14]:
df_transformed = df_clean \
        .withColumn('processing_delay', datediff(current_timestamp(), col('sourcets').cast('timestamp'))) \
        .withColumn('customer_status', expr("CASE WHEN customer_customerid IS NULL OR customer_customerid <> '' THEN FALSE ELSE TRUE END")) \
        .withColumn('local_stockcode', substring(col('lineitem_stockcode'), 0, 3)) \
        .withColumn('standard_stockcode', lpad(col('lineitem_stockcode'), 10, '0'))

Calculation started (calculation_id=96ca5e00-bf37-36bd-f44d-67c8b216d482) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [15]:
#Deduplication
window_dd = Window.partitionBy('region', 
                                'storepos', 
                                'storeno', 
                                'invoicedate', 
                                'invoiceno'
                                ).orderBy(desc('sourcets'))

df_deduplicated = df_transformed.withColumn('rownumber',row_number().over(window_dd))

# df_deduplicated.filter('rownumber>1').show(10)

# df_deduplicated.createOrReplaceTempView("df_dedup")

Calculation started (calculation_id=a4ca5e00-c473-eab1-e009-65407de9c04d) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [16]:
currency_convert = 's3://datacraft-retail-dataset/master-data/dim_currency_convert.csv'
currency = 's3://datacraft-retail-dataset/master-data/dim_currency.csv'
discount_code = 's3://datacraft-retail-dataset/master-data/dim_discount.csv'
tax_code = 's3://datacraft-retail-dataset/master-data/dim_geo_tax_code.csv'

Calculation started (calculation_id=ecca5e00-c980-5eea-2712-22a03f0c76a0) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [17]:
conversion_df = read_csv(currency_convert)
currency_df = read_csv(currency)
discount_df = read_csv(discount_code)
tax_df = read_csv(tax_code)

conversion_df.createOrReplaceTempView('conversion')
currency_df.createOrReplaceTempView('currency')
discount_df.createOrReplaceTempView('discount')
tax_df.createOrReplaceTempView('tax')

Calculation started (calculation_id=8eca5e00-ce8c-b04d-f118-7aca40feedae) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [18]:
df_deduplicated.createOrReplaceTempView("df_dedup")
# spark.sql("select * from df_dedup limit 5").show()

Calculation started (calculation_id=b6ca5e00-dfbe-1d43-1cbf-f955c12dfb6e) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [19]:
lookup_sql = '''select a.*, coalesce(b.currency_cd, "USD") as currency_code, coalesce(c.conversion_rate, 1) as conversion_rate, coalesce(d.discount_per, 5)/100 as discount_value
            from df_dedup a
            left join currency b on lower(a.storecountry) = lower(trim(b.country))
            left join conversion c on lower(trim(b.currency_cd)) = lower(trim(c.source_currency))
            left join discount d on lower(a.discountcode) = lower(trim(d.discount_cd))
            '''

df_lookup = spark.sql(lookup_sql)
# df_lookup.filter('region <> "Africa"').show(10)

Calculation started (calculation_id=c6ca5e00-e4d6-5fa4-8352-d961116dc6d1) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [20]:
# Calculations
txn_dataset = df_lookup.withColumn('lineitem_finalsaleprice', expr('ROUND((lineitem_saleprice-(lineitem_saleprice*discount_value))*conversion_rate, 2)')) \
                    .withColumn('create_ts', current_timestamp()) \
                    .withColumn('update_ts', current_timestamp()) \
                    .drop('rownumber','currency_code','conversion_rate','discount_value')

# txn_dataset.show(10)

Calculation started (calculation_id=a8ca5e00-e9e5-27c6-f8d9-0fc176b18ffc) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [21]:
txn_dataset.createOrReplaceTempView("txn_dataset")

Calculation started (calculation_id=bcca5e00-eee9-9db1-72de-da3b923cf5ae) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [28]:
agg_query = '''select  region, storeno, invoice_year, invoice_month,
                count(distinct(invoiceno)) as num_invoices, 
                count(distinct(lineitem_stockcode)) as stock_count,
                sum(lineitem_quantity) as total_quantity,
                sum(lineitem_finalsaleprice) as total_sales
            from txn_dataset
            group by region, storeno, invoice_year, invoice_month
            order by region, storeno, invoice_year desc, invoice_month desc
            '''

df_agg = spark.sql(agg_query)
df_agg.show(10)

Calculation started (calculation_id=68ca5e03-3de9-55b6-ae52-4999e9db4479) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------+-------+------------+-------------+------------+-----------+--------------+-----------+
|region|storeno|invoice_year|invoice_month|num_invoices|stock_count|total_quantity|total_sales|
+------+-------+------------+-------------+------------+-----------+--------------+-----------+
|Africa|      1|        2022|            3|           1|          1|          90.0|    5201.82|
|Africa|     11|        2023|            5|           1|          1|          24.0|     361.38|
|Africa|     12|        2020|            6|           1|          1|          97.0|    5838.62|
|Africa|     14|        2024|           10|           1|          1|          43.0|     2359.5|
|Africa|     14|        2020|           11|           1|          1|          39.0|     396.06|
|Africa|     15|        2024|           12|           1|          1|          48.0|    2399.47|
|Africa|     15|        2023|            1|           1|          1|           2.0|      52.52|
|Africa|     15| 

In [32]:
window_ll = Window.partitionBy('region', 'storeno') \
                    .orderBy(desc('invoice_year'))

df_leadlag = df_agg.withColumn('rownumber',row_number().over(window_ll)) \
                .withColumn('next_sales', lead(col('total_sales'),1).over(window_ll)) \
                .withColumn('prev_sales', lag(col('total_sales'),1).over(window_ll)) \
                .withColumn('change_pct', expr('ROUND((total_sales-prev_sales)/prev_sales*100, 2)'))

df_leadlag.show(10)

Calculation started (calculation_id=10ca5e05-0134-90d1-abd0-a5857fb17d5e) in (session=f0ca5e00-104e-b282-e34e-bc937991e894). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------+-------+------------+-------------+------------+-----------+--------------+-----------+---------+----------+----------+----------+
|region|storeno|invoice_year|invoice_month|num_invoices|stock_count|total_quantity|total_sales|rownumber|next_sales|prev_sales|change_pct|
+------+-------+------------+-------------+------------+-----------+--------------+-----------+---------+----------+----------+----------+
|Africa|      1|        2022|            3|           1|          1|          90.0|    5201.82|        1|      null|      null|      null|
|Africa|     11|        2023|            5|           1|          1|          24.0|     361.38|        1|      null|      null|      null|
|Africa|     12|        2020|            6|           1|          1|          97.0|    5838.62|        1|      null|      null|      null|
|Africa|     14|        2024|           10|           1|          1|          43.0|     2359.5|        1|    396.06|      null|      null|
|Afr

In [27]:
merge_condition = "target.invoiceno=updates.invoiceno and target.storeno=updates.storeno and target.lineitem_stockcode=updates.lineitem_stockcode and target.invoicedate=updates.invoicedate and target.storecountry=updates.storecountry"
agg_condition = "target.invoicedate=updates.invoicedate and target.storecountry=updates.storecountry"
partition_cols = ['region', 'invoicedate']
target_path = "s3://datacraft-retail-dataset/curated/txn_dataset/"

Calculation started (calculation_id=56ca53dc-1e8f-e79a-0188-a9aa17240794) in (session=64ca53db-37ac-5e13-b019-cdb042d19ab3). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [28]:
write_to_target(spark, txn_dataset, target_path, partition_cols, merge_condition)

Calculation started (calculation_id=98ca53dc-1faf-a7b9-9b18-f2b34583f82b) in (session=64ca53db-37ac-5e13-b019-cdb042d19ab3). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation 98ca53dc-1faf-a7b9-9b18-f2b34583f82b failed


  File "<stdin>", line 1, in <module>
NameError: name 'write_to_target' is not defined

